In [13]:
%matplotlib qt5
import kineticstoolkit.lab as ktk
import pandas as pd
import numpy as np
import matplotlib as plt
import c3d
import kineticstoolkit.geometry as geom

In [14]:
# reader = c3d.Reader(open("D:/Desktop/hackathon/move3d/3dmotion/20241021/c3d/T2G'002_20241021_Rehab_D4_07.c3d", 'rb'))

In [15]:
# for i, points, analog in reader.read_frames():
#     # print('frame {}: point {}, analog {}'.format(i, points.shape, analog.shape))
#     print(points.shape)
#     print(points)
#     # print('frame {}: point {}'.format(i, points.shape))

In [16]:
c3d_contents = ktk.read_c3d("D:\Desktop\hackathon\move3d/tests/trimmed_T2G002_20241021_Rehab_D4_01.c3d", convert_point_unit=True)

In [17]:
pts = c3d_contents["Points"]

dt = np.diff(pts.time).mean()    # average inter‐frame interval
print(dt)
fps = 1.0 / dt
print(f"Frame rate ≃ {fps:.2f} Hz")
# %matplotlib qt5

0.01
Frame rate ≃ 100.00 Hz


In [18]:
c3d_contents["Points"].data

{
          'LHIP': <array of shape (30252, 4)>
          'LKNE': <array of shape (30252, 4)>
          'LANK': <array of shape (30252, 4)>
          'RHIP': <array of shape (30252, 4)>
          'RKNE': <array of shape (30252, 4)>
          'RANK': <array of shape (30252, 4)>
          'RTOE': <array of shape (30252, 4)>
          'LTOE': <array of shape (30252, 4)>
         'Root3': <array of shape (30252, 4)>
    'RKneeAngle': <array of shape (30252, 4)>
          'Asym': <array of shape (30252, 4)>
}

In [19]:
markers = c3d_contents["Points"]
markers.data
# # # 2) define which labels to drop
# # to_drop = {"Root3", "RKneeAngle", "Asym"}

# # # 3) remove them from the data‐dict
# # for bad in to_drop:
# #     markers.data.pop(bad, None)

# # # 4) also update the labels list (so any UI reflects it)
# # if hasattr(markers, "labels"):
# #     markers.labels = [lbl for lbl in markers.labels if lbl not in to_drop]
# interconnections = {
#     "LHIP": {
#         "Color": [1, 0.25, 0],
#         "Links": [
#             ["LHIP", "LKNE"],
#             ["LKNE", "LANK"]
#         ],
#     },}
# # p = ktk.Player(markers, interconnections=interconnections)
# # p.play()

{
          'LHIP': <array of shape (30252, 4)>
          'LKNE': <array of shape (30252, 4)>
          'LANK': <array of shape (30252, 4)>
          'RHIP': <array of shape (30252, 4)>
          'RKNE': <array of shape (30252, 4)>
          'RANK': <array of shape (30252, 4)>
          'RTOE': <array of shape (30252, 4)>
          'LTOE': <array of shape (30252, 4)>
         'Root3': <array of shape (30252, 4)>
    'RKneeAngle': <array of shape (30252, 4)>
          'Asym': <array of shape (30252, 4)>
}

In [20]:
import io
import imageio
from tqdm import tqdm
import cv2
from vispy import scene, app

In [21]:
def get_img_from_fig(fig, dpi=120):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=dpi, bbox_inches="tight", pad_inches=0)
    buf.seek(0)
    img_arr = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    buf.close()
    img = cv2.imdecode(img_arr, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGBA)
    return img

In [22]:
import numpy as np
import imageio
from tqdm import tqdm
from vispy import scene, app

def motion2video_3d(
    motion,
    save_path,
    fps=25,
    keep_imgs=False,
    elevation=12,
    azimuth=80,
    zoom=1.0   # >1 zoom out, <1 zoom in
):
    if save_path.endswith(('/', '\\')):
        raise ValueError("save_path must include a filename, not end in '/'")

    # 1) Compute data bounds & center
    pts = motion.reshape(-1, 3)
    mins = pts.min(axis=0)
    maxs = pts.max(axis=0)
    c = (mins + maxs) / 2
    R0 = np.max(maxs - mins) / 2

    # 2) Grid half-range adjusts with zoom (symmetric around center)
    R_grid = R0 * zoom
    origin = c - R_grid  # grid starts at center - half-range
    coords_x = np.linspace(origin[0], origin[0] + 2*R_grid, 9)
    coords_y = np.linspace(origin[1], origin[1] + 2*R_grid, 9)
    coords_z = np.linspace(origin[2], origin[2] + 2*R_grid, 9)

    # 3) Video writer (GPU-accelerated NVENC)
    writer = imageio.get_writer(
        save_path,
        format='FFMPEG',
        mode='I',
        fps=fps,
        codec='h264_nvenc',
        ffmpeg_params=['-preset', 'fast', '-rc', 'vbr']
    )

    vlen = motion.shape[2]

    # 4) Offscreen VisPy canvas & camera, pivoting around the data center
    canvas = scene.SceneCanvas(show=False, size=(800, 800), bgcolor='white')
    view = canvas.central_widget.add_view()
    view.camera = scene.TurntableCamera(
        fov=45,
        center=tuple(c),         # pivot center
        distance=R0 * 5 * zoom,  # distance scales with zoom
        elevation=elevation,
        azimuth=azimuth
    )
    view.camera.up = '+z'

    # 5) Scaled & translated axes anchored at grid origin
    axis_scale = R_grid * 0.7
    axis = scene.visuals.XYZAxis(parent=view.scene)
    axis.transform = scene.transforms.STTransform(
        scale=(axis_scale,)*3,
        translate=tuple(origin)  # anchor axes at grid start
    )

    # 6) Labels just beyond each arrow tip at grid origin
    label_off = axis_scale * 1.1
    lbl_kw = dict(color='black', parent=view.scene, font_size=20, bold=True)
    scene.visuals.Text('X', pos=(origin[0] + label_off, origin[1], origin[2]), **lbl_kw)
    scene.visuals.Text('Y', pos=(origin[0], origin[1] + label_off, origin[2]), **lbl_kw)
    scene.visuals.Text('Z', pos=(origin[0], origin[1], origin[2] + label_off), **lbl_kw)

    # 7) Manual grids on all three planes (XY, XZ, YZ) symmetric around center
    grid_color = (0.6, 0.6, 0.6, 0.5)
    gw = 1.0
    # XY plane at z = origin[2]
    for x in coords_x:
        scene.visuals.Line(
            pos=np.array([[x, origin[1], origin[2]],
                          [x, origin[1] + 2*R_grid, origin[2]]], dtype=np.float32),
            color=grid_color, width=gw, parent=view.scene
        )
    for y in coords_y:
        scene.visuals.Line(
            pos=np.array([[origin[0], y, origin[2]],
                          [origin[0] + 2*R_grid, y, origin[2]]], dtype=np.float32),
            color=grid_color, width=gw, parent=view.scene
        )
    # # XZ plane at y = origin[1]
    # for x in coords_x:
    #     scene.visuals.Line(
    #         pos=np.array([[x, origin[1], origin[2]],
    #                       [x, origin[1], origin[2] + 2*R_grid]]], dtype=np.float32),
    #         color=grid_color, width=gw, parent=view.scene
    #     )
    # for z in coords_z:
    #     scene.visuals.Line(
    #         pos=np.array([[origin[0], origin[1], z],
    #                       [origin[0] + 2*R_grid, origin[1], z]]], dtype=np.float32),
    #         color=grid_color, width=gw, parent=view.scene
    #     )
    # # YZ plane at x = origin[0]
    # for y in coords_y:
    #     scene.visuals.Line(
    #         pos=np.array([[origin[0], y, origin[2]],
    #                       [origin[0], y, origin[2] + 2*R_grid]]], dtype=np.float32),
    #         color=grid_color, width=gw, parent=view.scene
    #     )
    # for z in coords_z:
    #     scene.visuals.Line(
    #         pos=np.array([[origin[0], origin[1], z],
    #                       [origin[0], origin[1] + 2*R_grid, z]]], dtype=np.float32),
    #         color=grid_color, width=gw, parent=view.scene
    #     )

    # 8) Prepare skeleton visuals
    scatter = scene.visuals.Markers(parent=view.scene)
    joint_pairs = [(0,1),(1,2),(0,3),(3,4),(4,5)]
    jp_left = [(0,3),(3,4),(4,5)]
    jp_right = [(0,1),(1,2)]
    colors = {
        'mid':   (0.00, 0.27, 0.49, 1.0),
        'left':  (0.01, 0.19, 0.37, 1.0),
        'right': (0.18, 0.44, 0.69, 1.0),
    }
    lines = [(limb, scene.visuals.Line(parent=view.scene, width=3, connect='segments'))
             for limb in joint_pairs]

    # 9) Render loop
    for f in tqdm(range(vlen), desc="Rendering frames"):
        j3d = motion[:, :, f]
        scatter.set_data(j3d, edge_color='black', face_color='white', size=6)
        for (a,b), ln in lines:
            seg = np.vstack([j3d[a], j3d[b]])
            color = (colors['left'] if (a,b) in jp_left
                     else colors['right'] if (a,b) in jp_right
                     else colors['mid'])
            ln.set_data(pos=seg, color=color)

        img = canvas.render()
        writer.append_data(img)
        if keep_imgs:
            imageio.imwrite(f"{save_path.split('.')[0]}_frame_{f:04d}.png", img)

    writer.close()


In [23]:
# 1) Define the exact joints your function will use (must match joint_pairs)
joint_names = [
    # 'Root3',  # hip‐center
    'RHIP', 'RKNE', 'RANK',
    'LHIP', 'LKNE', 'LANK',
]

# 2) Grab number of frames
n_frames = markers.data[joint_names[0]].shape[0]

# 3) Allocate the array (n_joints, 3, n_frames)
motion_array = np.zeros((len(joint_names), 3, n_frames), dtype=float)

# 4) Fill it: for each joint, take columns [0,1,2] (x,y,z) and transpose
for i, name in enumerate(joint_names):
    arr = markers.data[name]       # shape (n_frames, 4)
    motion_array[i, :, :] = arr[:, :3].T

# motion_array

In [24]:
output_file = r"D:/Desktop/hackathon/skeleton.mp4"
motion2video_3d(motion_array[:,:,:10000], output_file, fps=100, keep_imgs = False, elevation=12, azimuth=80-10, zoom=1.1)

Rendering frames: 100%|██████████| 10000/10000 [02:30<00:00, 66.55it/s]


In [239]:
def motion2video_3d(motion, save_path, fps=25, keep_imgs = False):
    if save_path.endswith(('/', '\\')):
        raise ValueError(f"save_path must include a filename, not end in '/': {save_path}")

    # videowriter = imageio.get_writer(save_path, fps=fps)
    videowriter = imageio.get_writer(
    save_path,
    format='FFMPEG',
    mode='I',
    fps=fps,
    codec='h264_nvenc',      # NVIDIA NVENC H.264 encoder
    ffmpeg_params=['-preset', 'fast', '-rc', 'vbr']  
    )
    vlen = motion.shape[2]
    save_name = save_path.split('.')[0]
    frames = []
    # joint_pairs = [
    #     ['Root3', 'RHIP'],
    #     ['RHIP',  'RKNE'],
    #     ['RKNE',  'RANK'],
    #     ['Root3', 'LHIP'],
    #     ['LHIP',  'LKNE'],
    #     ['LKNE',  'LANK'],
    # ]

    # # Right side only:
    # joint_pairs_right = [
    #     ['Root3', 'RHIP'],
    #     ['RHIP',  'RKNE'],
    #     ['RKNE',  'RANK'],
    # ]

    # # Left side only:
    # joint_pairs_left = [
    #     ['Root3', 'LHIP'],
    #     ['LHIP',  'LKNE'],
    #     ['LKNE',  'LANK'],
    # ]

    # joint_pairs       = [(0,1), (1,2), (2,3), (0,4), (4,5), (5,6)]
    # joint_pairs_right = [(0,1), (1,2), (2,3)]
    # joint_pairs_left  = [(0,4), (4,5), (5,6)]

    joint_pairs       = [(0,1), (1,2), (0,3), (3,4), (4,5)]
    joint_pairs_right = [(0,1), (1,2)]
    joint_pairs_left  = [(0,3), (3,4), (4,5)]
    
    color_mid = "#00457E"
    color_left = "#02315E"
    color_right = "#2F70AF"
    plt.ioff()
    for f in tqdm(range(vlen), desc="Rendering frames"):
        j3d = motion[:,:,f]
        fig = plt.figure(0, figsize=(10, 10))
        ax = plt.axes(projection="3d")
        ax.set_xlim(-1.5, 1.5)
        ax.set_ylim(-3, 0.5)
        ax.set_zlim(-1.5, 1.5)
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.view_init(elev=10., azim=80-100)
        plt.tick_params(left = False, right = False , labelleft = False ,
                        labelbottom = False, bottom = False)
        for i in range(len(joint_pairs)):
            limb = joint_pairs[i]
            xs, ys, zs = [np.array([j3d[limb[0], j], j3d[limb[1], j]]) for j in range(3)]
            if joint_pairs[i] in joint_pairs_left:
                ax.plot(xs, ys, zs, color=color_left, lw=3, marker='o', markerfacecolor='w', markersize=3, markeredgewidth=2) # axis transformation for visualization
            elif joint_pairs[i] in joint_pairs_right:
                ax.plot(xs, ys, zs, color=color_right, lw=3, marker='o', markerfacecolor='w', markersize=3, markeredgewidth=2) # axis transformation for visualization
            else:
                ax.plot(xs, ys, zs, color=color_mid, lw=3, marker='o', markerfacecolor='w', markersize=3, markeredgewidth=2) # axis transformation for visualization
            
        frame_vis = get_img_from_fig(fig)
        videowriter.append_data(frame_vis)
        plt.close()
    plt.ion()
    videowriter.close()

In [240]:
# # 1) Define the exact joints your function will use (must match joint_pairs)
# joint_names = [
#     # 'Root3',  # hip‐center
#     'RHIP', 'RKNE', 'RANK',
#     'LHIP', 'LKNE', 'LANK',
# ]

# # 2) Grab number of frames
# n_frames = markers.data[joint_names[0]].shape[0]

# # 3) Allocate the array (n_joints, 3, n_frames)
# motion_array = np.zeros((len(joint_names), 3, n_frames), dtype=float)

# # 4) Fill it: for each joint, take columns [0,1,2] (x,y,z) and transpose
# for i, name in enumerate(joint_names):
#     arr = markers.data[name]       # shape (n_frames, 4)
#     motion_array[i, :, :] = arr[:, :3].T

# # motion_array

In [241]:
# import matplotlib.pyplot as plt
# joint_pairs       = [(0,1), (1,2), (0,3), (3,4), (4,5)]
# joint_pairs_right = [(0,1), (1,2)]
# joint_pairs_left  = [(0,3), (3,4), (4,5)]

# color_mid = "#00457E"
# color_left = "#02315E"
# color_right = "#2F70AF"


# # plt.ion()
# j3d = motion_array[:,:,200]
# fig = plt.figure(0, figsize=(10, 10))
# ax = plt.axes(projection="3d")
# ax.set_xlim(-1.5, 1.5)
# ax.set_ylim(-1.5, 1.5)
# ax.set_zlim(-1.5, 1.5)
# ax.set_xlabel('X')
# ax.set_ylabel('Y')
# ax.set_zlabel('Z')
# ax.view_init(elev=12, azim=80-10)
# plt.tick_params(left = False, right = False , labelleft = False ,
#                 labelbottom = False, bottom = False)
# for i in range(len(joint_pairs)):
#     limb = joint_pairs[i]
#     xs, zs, ys = [np.array([j3d[limb[0], j], j3d[limb[1], j]]) for j in range(3)]
#     if joint_pairs[i] in joint_pairs_left:
#         ax.plot(-zs, xs, ys, color=color_left, lw=3, marker='o', markerfacecolor='w', markersize=3, markeredgewidth=2) # axis transformation for visualization
#     elif joint_pairs[i] in joint_pairs_right:
#         ax.plot(-zs, xs, ys, color=color_right, lw=3, marker='o', markerfacecolor='w', markersize=3, markeredgewidth=2) # axis transformation for visualization
#     else:
#         ax.plot(-zs, xs, ys, color=color_mid, lw=3, marker='o', markerfacecolor='w', markersize=3, markeredgewidth=2) # axis transformation for visualization
    
# frame_vis = get_img_from_fig(fig)
# plt.show()

In [242]:
motion_array.shape

(6, 3, 30252)

In [246]:
output_file = r"D:/Desktop/hackathon/skeleton.mp4"
motion2video_3d(motion_array[:,:,:10000], output_file, fps=100, keep_imgs = False)

Rendering frames:  31%|███▏      | 3133/10000 [05:19<11:41,  9.79it/s]


KeyboardInterrupt: 

## Base transformation

In [75]:
# 1) pick your 6 joints from markers.data
joint6_names = ['RHIP','RKNE','RANK','LHIP','LKNE','LANK']
N = markers.data[joint6_names[0]].shape[0]

# stack into a (6*N,3) array
P_list = [markers.data[name][:, :3] for name in joint6_names]  # each is (N,3)
P_orig = np.vstack(P_list)                                      # → (6N,3)

# 2) random similarity transform generator
def random_transform(seed=None):
    if seed is not None:
        np.random.seed(seed)
    # scale
    s = np.random.uniform(0.8, 1.2)
    # random unit quaternion
    u1,u2,u3 = np.random.rand(3)
    q = np.array([
        np.sqrt(1-u1)*np.sin(2*np.pi*u2),
        np.sqrt(1-u1)*np.cos(2*np.pi*u2),
        np.sqrt(u1)*np.sin(2*np.pi*u3),
        np.sqrt(u1)*np.cos(2*np.pi*u3),
    ])
    x,y,z,w = q
    R = np.array([
        [1-2*(y*y+z*z),   2*(x*y-z*w),     2*(x*z+y*w)],
        [2*(x*y+z*w),     1-2*(x*x+z*z),   2*(y*z-x*w)],
        [2*(x*z-y*w),     2*(y*z+x*w),     1-2*(x*x+y*y)]
    ])
    # translation
    t = np.random.uniform(-50, 50, size=3)

    T = np.eye(4)
    T[:3,:3] = s * R
    T[:3, 3] = t
    return T

# apply homogeneous transform
def apply_transform(pts, T):
    # pts: (M,3)
    M = pts.shape[0]
    pts_h = np.hstack([pts, np.ones((M,1))])   # (M,4)
    trans_h = (T @ pts_h.T).T                  # (M,4)
    return trans_h[:,:3]

# 3) build & apply
T_true  = random_transform(seed=42)
P_trans = apply_transform(P_orig, T_true)

In [76]:
# 4) Umeyama similarity (recover s,R,t)
def umeyama_similarity(P, Q, with_scaling=True):
    P = np.asarray(P); Q = np.asarray(Q)
    M = P.shape[0]
    mu_P = P.mean(axis=0)
    mu_Q = Q.mean(axis=0)
    P0 = P - mu_P
    Q0 = Q - mu_Q
    C  = (P0.T @ Q0) / M
    U, S, Vt = np.linalg.svd(C)
    D = np.eye(3)
    if (np.linalg.det(U) * np.linalg.det(Vt)) < 0:
        D[2,2] = -1
    R_est = U @ D @ Vt
    if with_scaling:
        varP = (P0**2).sum() / M
        s_est = np.trace(np.diag(S) @ D) / varP
    else:
        s_est = 1.0
    t_est = mu_Q - s_est * R_est @ mu_P
    # assemble homogeneous
    T_est = np.eye(4)
    T_est[:3,:3] = s_est * R_est
    T_est[:3, 3] = t_est
    return T_est, R_est, s_est, t_est

T_est, R_est, s_est, t_est = umeyama_similarity(P_orig, P_trans)

# 5) Compare
print("True T:\n", T_true)
print("\nEstimated T:\n", T_est)
print("\nMax element‐wise error:", np.max(np.abs(T_true - T_est)))

True T:
 [[  0.33910001  -0.84347953   0.27514359 -34.39813596]
 [  0.86448375   0.24786183  -0.30558612 -34.40054797]
 [  0.19957342   0.35952375   0.85619136 -44.19163878]
 [  0.           0.           0.           1.        ]]

Estimated T:
 [[  0.33910001   0.86448375   0.19957342 -32.50037136]
 [ -0.84347953   0.24786183   0.35952375 -34.81003981]
 [  0.27514359  -0.30558612   0.85619136 -44.91254221]
 [  0.           0.           0.           1.        ]]

Max element‐wise error: 1.8977645912753047


In [ ]:
# 1) Your six base joints (no root)
joint6_names = ['RHIP','RKNE','RANK','LHIP','LKNE','LANK']
#    the indices of these same joints in your 17-joint array:
idx17_for_6  = [1,     2,     3,     4,     5,     6    ]

# 2) Pull out motion6 and motion17 from your data dicts:
#    markers.data[name] has shape (N,4) so [:,:3] strips confidences.
N = markers.data['RHIP'].shape[0]

motion6 = np.stack([
    markers.data[name][:, :3].T   # (3, N) for each of the 6 joints
    for name in joint6_names
], axis=0)                        # → (6, 3, N)

motion17 = np.stack([
    other_data[name][:, :3].T     # your full 17-joint source
    for name in joint17_names
], axis=0)                        # → (17, 3, N)

# 3) Pick the “best” frame by summed confidence over those six joints:
confidence = np.stack([
    markers.data[name][:, 3]      # (N,) each
    for name in joint6_names
], axis=1)                         # → (N,6)
best_frame = int(np.argmax(confidence.sum(axis=1)))

# 4) Extract the two corresponding point sets at that frame:
#    P6: 6×3 points in basis-1;  Q6: 6×3 points in basis-2
P6 = motion6[:, :, best_frame].T                     # (6,3)
Q6 = motion17[idx17_for_6, :, best_frame]            # (6,3)

# 5) Compute the Umeyama similarity (R, t, s):
def umeyama_similarity(P, Q, with_scaling=True):
    P = np.asarray(P);  Q = np.asarray(Q)
    N = P.shape[0]
    mu_P = P.mean(axis=0)
    mu_Q = Q.mean(axis=0)
    P0 = P - mu_P
    Q0 = Q - mu_Q
    C  = (P0.T @ Q0) / N
    U, S, Vt = np.linalg.svd(C)
    D = np.eye(3)
    if (np.linalg.det(U) * np.linalg.det(Vt)) < 0:
        D[2,2] = -1
    R = U @ D @ Vt
    if with_scaling:
        varP = (P0**2).sum() / N
        s    = np.trace(np.diag(S) @ D) / varP
    else:
        s = 1.0
    t = mu_Q - s * R @ mu_P
    return R, t, s

R, t, s = umeyama_similarity(P6, Q6, with_scaling=True)

# 6) Build the 4×4 homogeneous transform:
T = np.eye(4)
T[:3,:3] = s * R
T[:3, 3] = t

# 7) Apply T to your entire 6-joint sequence in one go:
#    flatten all 6N points → transform → reshape back
P_all = motion6.transpose(2,0,1).reshape(-1,3)      # (6N,3)
P_h   = np.hstack([P_all, np.ones((P_all.shape[0],1))])  # (6N,4)
Q_h   = (T @ P_h.T).T                               # (6N,4)
Q_all = Q_h[:,:3].reshape(N, 6, 3).transpose(1,2,0) # → (6,3,N)

# Q_all now holds your 6-joint motion warped into the 17-point basis.

### Local origin construction

In [8]:
# origin = markers.data["LHIP"]
# y = markers.data["LHIP"] - markers.data["LKNE"]

# y_temp = y[:,:3]

# v = markers.data["LANK"] - markers.data["LHIP"]
# v_temp = v[:,:3]

# z = np.cross(y_temp, v_temp)
# z /= np.linalg.norm(z, axis=1, keepdims=True)

# x = np.cross(z, y_temp)
# x /= np.linalg.norm(x, axis=1, keepdims=True)

In [9]:
# import kineticstoolkit.geometry as geom

# # --- 2) Extract raw (N×4) marker arrays ---
# hip4   = markers.data["LHIP"]             # shape (N,4)
# knee4  = markers.data["LKNE"]
# ankle4 = markers.data["LANK"]

# # Build the three (N×3) direction/origin vectors
# O3  = hip4[:, :3]                         # hip origin
# Y3  = (hip4 - knee4)[:, :3]               # hip → knee (cranial)
# YZ3 = (hip4 - ankle4)[:, :3]              # hip → ankle (in‐plane)


# Y3 /= np.linalg.norm(Y3, axis=1)[:,None]
# YZ3 /= np.linalg.norm(YZ3, axis=1)[:,None]

# # --- 3) Promote to homogeneous (N×4) so that KTK sees “points” (w=1) vs “vectors” (w=0) ---
# N   = O3.shape[0]
# O4  = np.hstack([O3,  np.ones((N,1))])    # w=1 for positions
# Y4  = np.hstack([Y3,  np.zeros((N,1))])   # w=0 for direction vectors
# YZ4 = np.hstack([YZ3, np.zeros((N,1))])   # w=0 

# # --- 4) Build the Nx4×4 transform series (returns a raw ndarray) ---
# transforms = geom.create_transform_series(
#     x=None, y=Y4, z=None,  # define the primary axis via `y=`
#     xy=None, xz=None, yz=YZ4,
#     positions=O4,
#     length=N
# )  # → NumPy array of shape (N,4,4) :contentReference[oaicite:2]{index=2}

# # --- 5) Wrap into a KTK TimeSeries and merge with the markers ---
# frames = ktk.TimeSeries()                 
# frames.time = markers.time.copy()         # ensure same time‐base
# frames.data["FemurCS"] = transforms       # attach the Nx4×4 series

# # Merge and animate
# combined = markers.merge(frames)          # now contains both Points & the new FemurCS
# p = ktk.Player(combined, interconnections=interconnections)                  # see your markers + local femur CS together

In [15]:
# 2) Extract raw (N×4) marker arrays
hip4   = markers.data["LHIP"]
knee4  = markers.data["LKNE"]
ankle4 = markers.data["LANK"]

# 3) FEMUR CS: build 3D vectors (drop homogeneous col)
O_f3   = hip4[:, :3]                          # origin
Y_f3   = (hip4 - knee4)[:, :3]                # y axis (cranial)
YZ_f3  = (hip4 - ankle4)[:, :3]               # in‐plane vector

# normalize directions
def norm(v): return v / np.linalg.norm(v, axis=1, keepdims=True)
Y_f3, YZ_f3 = norm(Y_f3), norm(YZ_f3)

# promote to homogeneous (N×4)
N      = markers.time.shape[0]
O_f4   = np.hstack([O_f3,  np.ones((N,1))])   # w=1 for origin
Y_f4   = np.hstack([Y_f3,  np.zeros((N,1))])  # w=0 for direction
YZ_f4  = np.hstack([YZ_f3, np.zeros((N,1))])

# create femur transforms via y & yz → returns (N,4,4) ndarray :contentReference[oaicite:1]{index=1}
femur_tf = geom.create_transform_series(
    positions=O_f4,
    y=Y_f4,
    yz=YZ_f4,
    length=N
)

# 4) ANKLE CS **without** toe: use tibia axis + global vertical as plane vector
O_a3   = ankle4[:, :3]                        # origin at ankle
Y_a3   = norm((knee4 - ankle4)[:, :3])        # tibial long axis

# global vertical direction vector, homogeneous
Z0_4   = np.tile([0, 1, 0, 0], (N,1))         # w=0 for direction

O_a4   = np.hstack([O_a3,  np.ones((N,1))])
Y_a4   = np.hstack([Y_a3,  np.zeros((N,1))])

# create ankle transforms via y & yz=global vertical
ankle_tf = geom.create_transform_series(
    positions=O_a4,
    y=Y_a4,
    yz=Z0_4,
    length=N
)

# 5) Wrap each transform array in a ktk.TimeSeries
fem_ts = ktk.TimeSeries(); fem_ts.time = markers.time.copy()
fem_ts.data["FemurCS"] = femur_tf

ank_ts = ktk.TimeSeries(); ank_ts.time = markers.time.copy()
ank_ts.data["AnkleCS"] = ankle_tf

# 6) Merge markers + both CS and launch Player
combined = markers.merge(fem_ts).merge(ank_ts)
player   = ktk.Player(combined, interconnections=interconnections)  # interactive 3D animation :contentReference[oaicite:2]{index=2}

In [11]:
hip_to_ankle = ktk.geometry.get_local_coordinates(
    combined.data["FemurCS"], combined.data["AnkleCS"]
)

hip_to_ankle.shape 

(31543, 4, 4)

In [12]:
euler_angles = ktk.geometry.get_angles(hip_to_ankle, "ZXY")

np.array(euler_angles)

array([[0.10030886, 0.36116502, 2.86411627],
       [0.10012643, 0.35993664, 2.86374051],
       [0.09936711, 0.35351672, 2.86120751],
       ...,
       [0.12944888, 0.2402723 , 2.64102712],
       [0.12946875, 0.24050228, 2.64135675],
       [0.12929712, 0.24136394, 2.64339453]])

In [13]:
flexion_rad   = euler_angles[:, 0]
pronation_rad = euler_angles[:, 2]

flexion_unwrapped_rad   = np.unwrap(flexion_rad)
pronation_unwrapped_rad = np.unwrap(pronation_rad)

angles = ktk.TimeSeries(time=markers.time)

angles.data["Elbow flexion"] = flexion_unwrapped_rad
angles.data["Forearm pronation"] = pronation_unwrapped_rad

angles = angles.add_data_info("Elbow flexion", "Unit", "deg")
angles = angles.add_data_info("Forearm pronation", "Unit", "deg")

angles.plot()

## Angle Calculation

In [14]:
import numpy as np
import kineticstoolkit.geometry as geom
import matplotlib.pyplot as plt
# --- Helpers --------------------------------------------------
def normalize(v):
    return v / np.linalg.norm(v, axis=1, keepdims=True)

def make_homogeneous(points3):
    """Turn (N×3) into (N×4) with last col = 1."""
    N = points3.shape[0]
    return np.hstack([points3, np.ones((N,1))])

# --- 1) Synthetic marker data ---------------------------------
np.random.seed(0)
J = np.cumsum(np.random.randn(20, 3, 3) * 0.01, axis=0) + np.array([
    [0.0, 0.9, 0.0],  # hip
    [0.0, 0.5, 0.0],  # knee
    [0.0, 0.1, 0.0],  # ankle
])
hip3, knee3, ankle3 = J[:,0,:], J[:,1,:], J[:,2,:]
N = J.shape[0]

# --- 2) Femur CS via create_transform_series ---------------
#    y = hip→knee, yz = hip→ankle
O_f4  = make_homogeneous(hip3)
Y_f4  = np.hstack([normalize(hip3 - knee3),  np.zeros((N,1))])
YZ_f4 = np.hstack([normalize(hip3 - ankle3), np.zeros((N,1))])

T_fem = geom.create_transform_series(
    positions=O_f4,
    y=Y_f4,
    yz=YZ_f4,
    length=N
)

# --- 3) Ankle CS via create_transform_series --------------
#    y = knee→ankle, yz = global vertical
O_a4  = make_homogeneous(ankle3)
Y_a4  = np.hstack([normalize(knee3 - ankle3), np.zeros((N,1))])
Z0_4  = np.tile([0,1,0,0], (N,1))  # w=0 for a direction vector

T_ank = geom.create_transform_series(
    positions=O_a4,
    y=Y_a4,
    yz=Z0_4,
    length=N
)

# --- 4) Local transforms & orthogonality check ------------
#    (femur in ankle frame)
T_local = geom.get_local_coordinates(T_fem, T_ank)
# Internally this checks each R·Rᵀ = I, so no left-handed det issues

# --- 5) Euler angles (intrinsic ZXY) -----------------------
euler = geom.get_angles(T_local, seq="ZXY", degrees=True, flip=False)
# euler.shape == (N,3)

# --- 6) Unwrap to remove ±180° jumps ----------------------
# euler_unw = np.rad2deg(np.unwrap(np.deg2rad(euler), axis=0))
print(euler_unw)  # (N,3)
# --- 7) Plot -------------------------------------------------
time = np.arange(N) / 50.0  # assume 50 Hz sampling
labels = ["Z (Ab/Adduction)", "X (Flex/Extension)", "Y (Int/Ext Rotation)"]

plt.figure(figsize=(8,4))
for i in range(3):
    plt.plot(time, euler[:,i], label=labels[i])
plt.xlabel("Time (s)")
plt.ylabel("Angle (°)")
plt.legend()
plt.title("Synthetic Hip-to-Ankle Joint Angles via KTK")
plt.tight_layout()
plt.show()

NameError: name 'euler_unw' is not defined